## SVM

In [23]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
import matplotlib.pyplot as plt
import timeit

In [24]:
from preprocessing import cargarDatasets
from preprocessing import prepararSetDeEntrenamiento
from preprocessing import prepararSetDeValidacion
from preprocessing import ingenieriaDeFeaturesSVM

In [25]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score

In [26]:
from sklearn.svm import SVC

* Este metodo es un algortimo suvervisado en la cual busca un hiperplano que separae las clases de la mejor forma posible. 
* A su vez busca maximinar M (margen), el espacio entre los distintos puntos de las distintas clases. 
  Si se decea que los punos esten perfectamente separados tenemos un modelo hard margin, en donde el modelo no funciona con outliers y ruido. Si se le permite tener algo de rido y outliers entonces tenemos un modelo soft margin.

* En el caso en que no se encuentre el hiperplano, los puntos no son linealmente separables por lo que hay reccurir a una transformacion de datos en una nueva dimension. Como es costoso se utilizan kernes que hace que se piense que esta en otro dimension. Estos kernel puede sen lineal, polinomial, radial, etc.

## Carga y preprosesamiento de datos 

In [27]:
train_df,final_df = cargarDatasets()
train_df = prepararSetDeEntrenamiento(train_df)


Una vez que tenemos cargados el set de entrenamiento vamos a ir aplicando distintos preprocesamientos

svm es sensible a la escala y distribucion de datos, es por eso que normalizamos las variables.

In [28]:
X,y,df,y_encoder = ingenieriaDeFeaturesSVM(train_df)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=117, test_size=0.1, stratify=y)

## Busca de hiperparaemtros

los hiperparametros a tener en cuenta van a ser c y el kernel 
* c: es la resistencia al ruido, a mayor c se tiene un modelo hard margin y a menor c se tiene un modelo soft margin.
* kernel: es una funcion que nos permite tener dimensiones mayores sin tener que convertir los datos.     

In [30]:
parametros_lineal = [ {'C': [1,2,3] , 'kernel': ['linear']}]

In [31]:
parametros_radial = [ {'C': [0.5,1,2,3], 'gamma': [0.0000001,0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 'kernel': ['rbf']}]

In [40]:
parametro_polinomial = [{'C': [1,2,3], 'gamma': [0.0000001,0.0001,0.01, 0.1, 1], 'degree':[3,10,100,500]}]

In [33]:
svm = SVC(probability = True)
lineal = RandomizedSearchCV(
    svm, parametros_lineal, scoring='roc_auc', n_jobs=-1, cv=2, return_train_score=True, n_iter = 8, verbose = 99
)

In [34]:
resultado1 = lineal.fit(X_train, y_train)

Fitting 2 folds for each of 3 candidates, totalling 6 fits


/home/agus/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 3 is smaller than n_iter=8. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [35]:
print('mejor score: %s' % resultado1.best_score_)
print('mejores hiperparametros %s' % resultado1.best_params_)
   

mejor score: 0.8816213603693884
mejores hiperparametros {'kernel': 'linear', 'C': 1}


Pruebo con radial

In [36]:
svm = SVC(probability = True)
radial = RandomizedSearchCV(
    svm, parametros_radial, scoring='roc_auc', n_jobs=-1, cv=2, return_train_score=True, n_iter = 8, verbose = 99
)

In [37]:
resultado2 = radial.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits


In [38]:
print('mejor score: %s' % resultado2.best_score_)
print('mejores hiperparametros %s' % resultado2.best_params_)

mejor score: 0.8795917583695099
mejores hiperparametros {'kernel': 'rbf', 'gamma': 0.001, 'C': 1}


Pruebo con polonimial

In [41]:
svm = SVC(probability = True)
poly = RandomizedSearchCV(
    svm, parametro_polinomial, scoring='roc_auc', n_jobs=-1, cv=3, return_train_score=True, n_iter = 8, verbose = 99
)

In [42]:
resultado3 = poly.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


In [43]:
print('mejor score: %s' % resultado3.best_score_)
print('mejores hiperparametros %s' % resultado3.best_params_)

mejor score: 0.8776762962876234
mejores hiperparametros {'gamma': 1e-07, 'degree': 10, 'C': 3}


In [ ]:
vemos que el mejoor score lo obtuvimos con el kernel lineal. Ahora entrenamos.

# Entrenamos Modelo

In [47]:
svm = SVC(C = 1, kerner = 'linear')
svm.fit(X_train, y_train)

TypeError: __init__() got an unexpected keyword argument 'kerner'